In [1]:
import cv2
from PIL import Image
import torch

In [2]:
def start_streaming():
    cap = cv2.VideoCapture(0)  # Use 0 for the default camera
    # Set up window
    window_name = "Scene Description Stream"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break


            # Overlay the description on the frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, "hello", (10, 50), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display the frame
            cv2.imshow(window_name, frame)

            # Exit on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

In [3]:
start_streaming()

: 